# Probabilistic forecasting I: Temperature
## Kaggle Competition hosted by Carl McBride Ellis
## Catboost Multiquantiles Regression - Conformalized Quantile Regression

Submission Score: xxxxxx

### https://www.linkedin.com/in/stephane-degeye-b460a5221/

# Some Librairies

In [ ]:
!pip install catboost

In [ ]:
import utility_script_crps_score as us
# help(us.crps)

# Data Structure
import numpy as np
import pandas as pd

# Model
from catboost import CatBoostRegressor

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

# Utils
from sklearn.model_selection import train_test_split

# Error Handling
import warnings
warnings.filterwarnings("ignore")

# Plot
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

plt.style.use('ggplot')
plt.rcParams.update(**{'figure.dpi':150})

%matplotlib inline

# A. Data Loading

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# # Folder related to n-beats data
# folder_id = 'kaggle/probabilistic-forecasting-i-temperature'

# # Full path
# folder_path = '/content/gdrive/My Drive/'+ folder_id

# train_df = pd.read_csv(folder_path + '/dataset/train.csv')
# kaggle_test_df = pd.read_csv(folder_path + '/dataset/test.csv')
# submission_df = pd.read_csv(folder_path + '/submission/sample_submission.csv')

In [ ]:
# Load the dataset
train_df = pd.read_csv('/kaggle/input/probabilistic-forecasting-i-temperature/train_and_Public.csv')
kaggle_test_df = pd.read_csv('/kaggle/input/probabilistic-forecasting-i-temperature/test.csv')
submission_df = pd.read_csv('/kaggle/input/probabilistic-forecasting-i-temperature/sample_submission.csv')

In [ ]:
train_df

In [ ]:
kaggle_test_df

# B. Data Preparation

In [ ]:
train_df['date'] = pd.to_datetime(train_df['date'])

In [ ]:
kaggle_test_df['date'] = pd.to_datetime(kaggle_test_df['date'])

# C. Exploratory Data Analysis

## Shape of Dataframe

In [ ]:
num_rec_train = train_df.shape[0]
print(f'Train dataframe contains {num_rec_train} records belongings to {round(num_rec_train / 96, 1)} days')

num_rec_test = kaggle_test_df.shape[0]
print(f'Test dataframe contains {num_rec_test} records belongings to {round(num_rec_test / 96, 1)} days')

## Missing Values

In [ ]:
# Detecting missing values in train_df
missing_values = train_df.isnull().sum()

print("Missing values in each column of train_df:")
print(missing_values)

# Detecting missing values in kaggle_test_df
missing_values = kaggle_test_df.isnull().sum()

print("Missing values in each column of kaggle_test_df:")
print(missing_values)

## Tabular View

In [ ]:
train_df.head()

In [ ]:
kaggle_test_df.head()

## Data Distribution

In [ ]:
# Set the theme for the plot
sns.set_theme(style="whitegrid")

# Create a histogram of the total bill
plt.figure(figsize=(10, 6))
sns.histplot(train_df['Temperature'], bins=30, kde=True, color='blue')

# Adding titles and labels
plt.title('Distribution of Temperature', fontsize=16)
plt.xlabel('Temperature', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Show the plot
plt.show()

## Check Stationarity

In [ ]:
ADF_result = adfuller(train_df['Temperature'])

print(f'ADF Statistic: {ADF_result[0]}')
print(f'p-value: {ADF_result[1]}')

<div class="alert alert-block alert-danger">
<b> [Stationarity] </b> p-value < 0.05, so we can reject the Null Hypothesis, Time Series is Stationary!</div>

## Partial Autocorrelation

In [ ]:
plot_pacf(train_df['feature_AA'], lags=100)
plt.ylim(-0.2, 1.1)
plt.tight_layout

# Adding titles and labels
plt.title('Partial Autocorrelation feature_AA', fontsize=16)

In [ ]:
plot_pacf(train_df['feature_AB'], lags=100)
plt.ylim(-0.2, 1.1)
plt.tight_layout

# Adding titles and labels
plt.title('Partial Autocorrelation feature_AB', fontsize=16)

In [ ]:
plot_pacf(train_df['feature_BA'], lags=100)
plt.ylim(-0.2, 1.1)
plt.tight_layout

# Adding titles and labels
plt.title('Partial Autocorrelation feature_BA', fontsize=16)

In [ ]:
plot_pacf(train_df['feature_BB'], lags=100)
plt.ylim(-0.2, 1.1)
plt.tight_layout

# Adding titles and labels
plt.title('Partial Autocorrelation feature_BB', fontsize=16)

In [ ]:
plot_pacf(train_df['feature_CA'], lags=100)
plt.ylim(-0.2, 1.1)
plt.tight_layout

# Adding titles and labels
plt.title('Partial Autocorrelation feature_CA', fontsize=16)

In [ ]:
plot_pacf(train_df['feature_CB'], lags=100)
plt.ylim(-0.2, 1.1)
plt.tight_layout

# Adding titles and labels
plt.title('Partial Autocorrelation feature_CB', fontsize=16)

# D. Features Engineering

In [ ]:
# Inform model about the temporal progression of the data
train_df['id'] = train_df['id'].astype(float)
kaggle_test_df['id'] = kaggle_test_df['id'].astype(float)

# Create Calendar Features
train_df['year'] = train_df['date'].dt.year.astype(int)
train_df['month'] = train_df['date'].dt.month.astype(int)
train_df['day'] = train_df['date'].dt.day.astype(int)
train_df['hour'] = train_df['date'].dt.hour.astype(int)
train_df['week_of_year'] = train_df['date'].apply(lambda x: x.isocalendar()[1])
train_df['day_of_week'] = train_df['date'].dt.dayofweek.astype(int)

kaggle_test_df['year'] = kaggle_test_df['date'].dt.year.astype(int)
kaggle_test_df['month'] = kaggle_test_df['date'].dt.month.astype(int)
kaggle_test_df['day'] = kaggle_test_df['date'].dt.day.astype(int)
kaggle_test_df['hour'] = kaggle_test_df['date'].dt.hour.astype(int)
kaggle_test_df['week_of_year'] = kaggle_test_df['date'].apply(lambda x: x.isocalendar()[1])
kaggle_test_df['day_of_week'] = kaggle_test_df['date'].dt.dayofweek.astype(int)

# Create Lag Features
features_prefix = "feature_"
features_suffix = "_lag_"

lag_features = [
                {'AA' : ['1', '2', '4', '6', '97', '98']},
                {'AB' : ['1', '2', '3', '98']},
                {'CA' : ['1', '36', '97', '98']},
                {'CB' : ['1', '2', '3']}
               ]

for lag_feature in lag_features:
    for lag_list in lag_feature.values():    
        for lag in lag_list:
          train_df[features_prefix + ', '.join(lag_feature.keys()) + features_suffix + lag] = train_df[features_prefix + ', '.join(lag_feature.keys())].shift(int(lag))
          kaggle_test_df[features_prefix + ', '.join(lag_feature.keys()) + features_suffix + lag] = kaggle_test_df[features_prefix + ', '.join(lag_feature.keys())].shift(int(lag))

# Create Fourier terms for dayly seasonality
train_fourier_day = np.sin(2 * np.pi * np.arange(len(train_df)) / 96)
train_fourier_day += np.cos(2 * np.pi * np.arange(len(train_df)) / 96)
train_df['fourier_terms_day'] = train_fourier_day

kaggle_test_fourier_day = np.sin(2 * np.pi * np.arange(len(kaggle_test_df)) / 96)
kaggle_test_fourier_day += np.cos(2 * np.pi * np.arange(len(kaggle_test_df)) / 96)
kaggle_test_df['fourier_terms_day'] = kaggle_test_fourier_day

# Create Fourier terms for weekly seasonality
train_fourier_week = np.sin(2 * np.pi * np.arange(len(train_df)) / 96*7)
train_fourier_week += np.cos(2 * np.pi * np.arange(len(train_df)) / 96*7)
train_df['fourier_terms_week'] = train_fourier_week

kaggle_test_fourier_week = np.sin(2 * np.pi * np.arange(len(kaggle_test_df)) / 96*7)
kaggle_test_fourier_week += np.cos(2 * np.pi * np.arange(len(kaggle_test_df)) / 96*7)
kaggle_test_df['fourier_terms_week'] = kaggle_test_fourier_week

# Remove highly correlated features (unnecessarily increase model complexity)
train_df.drop('feature_BA', axis=1, inplace=True)
train_df.drop('feature_BB', axis=1, inplace=True)
train_df.drop('date', axis=1, inplace=True)

kaggle_test_df.drop('feature_BA', axis=1, inplace=True)
kaggle_test_df.drop('feature_BB', axis=1, inplace=True)
kaggle_test_df.drop('date', axis=1, inplace=True)

In [ ]:
train_df

In [ ]:
kaggle_test_df

## Pearson Pairwise Correlation

In [ ]:
pear_corr = train_df.drop(['id'], axis=1).corr(method='pearson')
pear_corr.style.background_gradient(cmap='Greens', axis=0)

# E. Modeling

## Hyperparameters

In [ ]:
quantile_mapping = {
    95: ['0.025', '0.975'],
    90: ['0.05', '0.95'],
    80: ['0.10', '0.90'],
    70: ['0.15', '0.85'],
    60: ['0.20', '0.80'],
    50: ['0.25', '0.75'],
    40: ['0.30', '0.70'],
    30: ['0.35', '0.65'],
    20: ['0.40', '0.60'],
    10: ['0.45', '0.55']
}

In [ ]:
quantiles = [0.025, 0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,0.975]
levels = np.array([95, 90, 80, 70, 60, 50, 40, 30, 20, 10])

In [ ]:
quantile_str = str(quantiles).replace('[','').replace(']','')

In [ ]:
submission_val_df = submission_df[:5360].copy()

## Dataset Splitting

In [ ]:
X = train_df.drop('Temperature', axis=1)
y = train_df['Temperature']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=5360)

In [ ]:
X_proper_train, X_cal, y_proper_train, y_cal = train_test_split(X_train, y_train, test_size=0.12, random_state=1)

## Model Definition and Fitting on Proper Training dataset

In [ ]:
# model definition
multiquantiles_model = CatBoostRegressor(
                                        loss_function = f'MultiQuantile:alpha={quantile_str}',
                                        thread_count = -1,
                                        bootstrap_type =  "Bernoulli",
                                        sampling_frequency= 'PerTree',
                                        iterations = 10000,
                                     **{'learning_rate': 0.015,
                                          'max_depth': 10,
                                          'subsample': 0.75,
                                          'colsample_bylevel': 0.9,
                                          'min_data_in_leaf': 37},
                                           verbose=1000
                              )


# model fitting on training set
multiquantiles_model.fit(X_proper_train, y_proper_train, eval_set=(X_val, y_val))

## Predictions

In [ ]:
predict_cal = multiquantiles_model.predict(X_cal)
predict_val = multiquantiles_model.predict(X_val)
predict_test = multiquantiles_model.predict(kaggle_test_df)

# F. Conformalized Quantile Regression (CQR)

In [ ]:
for index, (level, q_pair) in enumerate(quantile_mapping.items()):

    print(f'level : {level} interval : [{q_pair[0]} , {q_pair[1]}]\n')

    # Keep only predicted Quantiles for the calibration set related to the current level
    quantile_regression_calibration_intervals = np.zeros([len(X_cal), 2])
    quantile_regression_calibration_intervals[:, 0] = predict_cal[:, index]
    quantile_regression_calibration_intervals[:, 1] = predict_cal[:, 20 - index]

    # Compute Non-Conformity Measures on the Calibration set (How predicted Quantiles relate to the True value)
    non_conformity_scores = np.max(
    [
        quantile_regression_calibration_intervals[:, 0] - y_cal,
        y_cal - quantile_regression_calibration_intervals[:, 1],
    ],
    axis=0,
    )

    # Sort Non-Conformity Measures
    non_conformity_scores = np.sort(conformity_scores)[::-1]
    
    # Compute the Quantile based on Non-Conformity Measures distribution with level as threshold
    emperical_quantile = (level/100) * (1 + (1 / len(y_cal)))
    correction_factor = np.quantile(non_conformity_scores, emperical_quantile, method="higher")
    
    # Plot the non_conformity_scores distribution
    plt.hist(conformity_scores, bins='auto', color='magenta')

    # Add a vertical line for the
    plt.axvline(correction_factor, color='black', linestyle='dashed', linewidth=1, label='quantile')

    plt.legend()
    plt.xlabel('Calibration Error')
    plt.ylabel('Frequency')
    plt.title('Histogram of Calibration Errors')

    plt.show()

    # Keep only predicted Quantiles for the Kaggle test set related to the current level
    quantile_regression_prediction_intervals_kaggle = np.zeros([len(kaggle_test_df), 2])
    quantile_regression_prediction_intervals_kaggle[:, 0] = predict_test[:, index]
    quantile_regression_prediction_intervals_kaggle[:, 1] = predict_test[:, 20 - index]

    # Apply Correction factor on Kaggle test set
    correction_factor_kaggle = np.ones([len(kaggle_test_df), 2])
    correction_factor_kaggle[:, 0] *= correction_factor
    correction_factor_kaggle[:, 1] *= correction_factor
    
    submission_df[q_pair[0]] = quantile_regression_prediction_intervals_kaggle[:, 0] - correction_factor_kaggle[:, 0]
    submission_df[q_pair[1]] = quantile_regression_prediction_intervals_kaggle[:, 1] + correction_factor_kaggle[:, 1]

    # Keep only predicted Quantiles for the Validation set related to the current level
    quantile_regression_prediction_intervals_validation = np.zeros([len(X_val), 2])
    quantile_regression_prediction_intervals_validation[:, 0] = predict_val[:, index]
    quantile_regression_prediction_intervals_validation[:, 1] = predict_val[:, 20 - index]

    # Apply Correction factor on Validation set
    correction_factor_validation = np.ones([len(X_val), 2])
    correction_factor_validation[:, 0] *= correction_factor
    correction_factor_validation[:, 1] *= correction_factor
  
    submission_val_df[q_pair[0]] = quantile_regression_prediction_intervals_validation[:, 0] - correction_factor_validation[:, 0]
    submission_val_df[q_pair[1]] = quantile_regression_prediction_intervals_validation[:, 1] + correction_factor_validation[:, 1]

In [ ]:
## Median
submission_df['0.50'] = predict_test[:, 10]
submission_val_df['0.50'] = predict_val[:, 10]

In [ ]:
submission_df

# G. Prediction Interval (PI) Visualization

In [ ]:
for (level, q_pair) in quantile_mapping.items():

    print(f'level : {level} interval : [{q_pair[0]} , {q_pair[1]} ]\n')
    
    title = 'interval : ' + q_pair[0] + ' - ' + q_pair[1]
    
    # Create a new figure
    plt.figure(figsize=(10, 6))

    # Plot actual values
    plt.plot(submission_df['0.50'], label='median', color='blue')


    # Plot prediction intervals
    plt.fill_between(submission_df.index, submission_df[q_pair[0]], submission_df[q_pair[1]],
                     color='gray', alpha=0.2, label='Prediction Interval')

    # Add the legend
    plt.legend()

    # Set labels and title
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.title(title)

    # Show the plot
    plt.show()

# E. Evaluation

In [ ]:
# Dataframe with results
y_val = pd.DataFrame(y_val, columns=['Temperature'])
val_df = pd.concat([X_val, y_val], axis=1)

In [ ]:
us.crps(submission_val_df, val_df)

In [ ]:
us.coverage_report(submission_val_df, val_df)

# F. Save Submission to file

In [ ]:
submission_df.to_csv('submission.csv', index=False)

# End of code